# LA County coronavirus vaccine administration by city/community

### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_latimes as lat
import glob
import path
import os

In [3]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Create a timeseries from coronavirus-tracker data

In [4]:
path = "/Users/mhustiles/data/github/coronavirus-tracker/_notebooks/data/raw/vaccines/los-angeles/"
all_files = glob.glob(os.path.join(path, "*.csv"))

In [5]:
df_from_each_file = (
    pd.read_csv(f, encoding="ISO-8859-1", low_memory=False) for f in all_files
)
concatenated_df = pd.concat(df_from_each_file, ignore_index=True)

In [6]:
df = concatenated_df.copy()

In [7]:
population = df[df["update_date"] == df["update_date"].max()][["area", "pop_2019"]]

In [8]:
population.head()

,area,pop_2019
4176,City of Agoura Hills,18019
4177,City of Alhambra,72940
4178,City of Arcadia,51103
4179,City of Artesia,14266
4180,City of Avalon,3077


### Strip out the strings placed in columns by the county

In [9]:
df = df[
    (df.vaccinations.str.strip() != "Unreliable Data")
    & (df.pop_2019.str.strip() != "No Pop Data")
    & (df.vaccinations_pct.str.strip() != "Unreliable Data")
    & (df.vaccinations_pct.str.strip() != "No Pop Data")
    & (df.vaccinations.str.strip() != "<5")
]

### Merge with population figures/rates from vaccine disparity project

In [10]:
disparity = pd.read_csv("../../vaccine-disparity/_workspace/neighborhoods_grouped.csv")

In [11]:
disparity_clean = disparity[
    [
        "name",
        "total_pop",
        "totalpop_17andunder",
        "totalpop_18to64",
        "totalpop_65andup",
        "totalpop_17andunder_percent",
        "totalpop_65andup_percent",
        "totalpop_65andup_percent",
        "majority_white",
        "majority_hispanic",
        "median_householdincome",
    ]
]

In [12]:
df_pop_merge = pd.merge(
    disparity_clean, df, left_on="name", right_on="area", indicator=True
)

In [13]:
df_pop_merge._merge.value_counts()

both          11924
right_only        0
left_only         0
Name: _merge, dtype: int64

In [14]:
df["week"] = pd.to_datetime(df["update_date"]).dt.week

In [15]:
df.week.value_counts()

13    2352
12    2352
11    2352
10    2342
14    2340
9     2338
8     1670
15     332
Name: week, dtype: int64

In [16]:
df.sort_values("update_date", ascending=False).head(10)

,area,vaccinations,pop_2019,vaccinations_pct,update_date,week
4259,City of West Covina,38583,88873,43.4,2021-04-12,15
4406,Unincorporated - Angeles National Forest,296,1096,27.0,2021-04-12,15
4404,Unincorporated - Altadena,17205,36812,46.7,2021-04-12,15
4403,Unincorporated - Agua Dulce,958,3736,25.6,2021-04-12,15
4402,Unincorporated - Acton,1652,6961,23.7,2021-04-12,15
4401,Los Angeles - Woodland Hills,27387,57284,47.8,2021-04-12,15
4400,Los Angeles - Winnetka,15821,41471,38.1,2021-04-12,15
4399,Los Angeles - Wilshire Center,13370,40599,32.9,2021-04-12,15
4398,Los Angeles - Wilmington,11236,42449,26.5,2021-04-12,15
4397,Los Angeles - Wholesale District,12332,32528,37.9,2021-04-12,15


In [17]:
df["area"] = df["area"].str.replace("City of", "", regex=False)

### Clean up data types

In [18]:
df["vaccinations"] = df["vaccinations"].astype(float)

In [19]:
df["pop_2019"] = df["pop_2019"].astype(float)

In [20]:
df["vaccinations_pct"] = ((df["vaccinations"] / df["pop_2019"]) * 100).round(2)

### Which week of the year? 

In [21]:
df["week"] = pd.to_datetime(df["update_date"]).dt.week

In [22]:
df["weekday"] = pd.to_datetime(df["update_date"]).dt.day_name()

### Just get the updates each Monday

In [23]:
df = df[df["weekday"] == "Monday"]

In [24]:
df[df["area"].str.contains("Culver")].sort_values("update_date", ascending=True).head()

,area,vaccinations,pop_2019,vaccinations_pct,update_date,week,weekday
14637,Culver City,7445.0,32840.0,22.67,2021-03-01,9,Monday
6981,Culver City,7445.0,32840.0,22.67,2021-03-08,10,Monday
15333,Culver City,9969.0,32840.0,30.36,2021-03-15,11,Monday
5937,Culver City,9969.0,32840.0,30.36,2021-03-22,12,Monday
14289,Culver City,13819.0,32840.0,42.08,2021-03-29,13,Monday


### Widen the data for weekly average columns

In [25]:
df_pivot = (
    pd.pivot_table(
        df,
        values="vaccinations_pct",
        index="area",
        columns="week",
        aggfunc="mean",
    )
    .round(2)
    .reset_index()
)

In [26]:
df_pivot.head()

week,area,9,10,11,12,13,14,15
0,Agoura Hills,21.82,21.82,28.70,28.70,38.91,38.91,47.18
1,Alhambra,18.68,18.68,26.57,26.57,37.35,37.35,45.10
2,Arcadia,20.90,20.90,27.74,27.74,37.66,37.66,46.08
3,Artesia,20.83,20.83,25.73,25.73,35.27,35.27,42.27
4,Avalon,10.00,10.00,10.57,10.57,10.27,10.27,10.43


In [27]:
df_pivot.columns = ["pct_week_" + str(col) for col in df_pivot.columns]

In [28]:
df_pivot.rename(
    columns={"pct_week_area": "name", "week_pop_2019": "population"}, inplace=True
)

In [29]:
df_pivot.head()

,name,pct_week_9,pct_week_10,pct_week_11,pct_week_12,pct_week_13,pct_week_14,pct_week_15
0,Agoura Hills,21.82,21.82,28.70,28.70,38.91,38.91,47.18
1,Alhambra,18.68,18.68,26.57,26.57,37.35,37.35,45.10
2,Arcadia,20.90,20.90,27.74,27.74,37.66,37.66,46.08
3,Artesia,20.83,20.83,25.73,25.73,35.27,35.27,42.27
4,Avalon,10.00,10.00,10.57,10.57,10.27,10.27,10.43


### Merge with population figures/rates from vaccine disparity project

In [30]:
disparity = pd.read_csv("../../vaccine-disparity/_workspace/neighborhoods_grouped.csv")

In [31]:
disparity.head()

,Unnamed: 0,id,name,tract,tract_percent,Unnamed: 0_x,total_white,white_17andunder,white_18to64,white_65andup,total_hispanic,hispanic_17andunder,hispanic_18to64,hispanic_65andup,total_black,black_17andunder,black_18to64,black_65andup,total_native_american,native_american_17andunder,native_american_18to64,native_american_65andup,total_other,other_17andunder,other_18to64,...,total_asian_percent,asian_17andunder_percent,asian_18to64_percent,asian_65andup_percent,total_pacificislander_percent,pacificislander_17andunder_percent,pacificislander_18to64_percent,pacificislander_65andup_percent,nonwhite_percentage,totalpop_17andunder_percent,totalpop_18to64_percent,totalpop_65andup_percent,healthcare_workers_percent,majority_nonwhite,majority_black,majority_asian,majority_white,majority_american_indian_and_alaska_native,majority_hispanic,majority_18to64,majority_65andup,Unnamed: 0_y,neighborhood,median_householdincome,majorityhigherthanmedianincomeLA
0,0,1,Agoura Hills,56022.97,3.015807,15128,14294.386712,2565.000554,9240.747705,2488.638453,2060.384429,637.837458,1224.501950,0.000021,475.768824,167.211468,235.017585,73.539771,29.800880,16.466653,13.334227,0.000000,1360.190418,599.562387,692.685050,...,0.084100,0.076485,0.085944,0.084423,0.000000,0.000000,0.000000,0.000000,0.256824,0.215044,0.630619,0.159371,0.058344,True,False,False,True,False,False,True,False,0.0,Agoura Hills,121292.28270,True
1,1,2,Alhambra,161868.09,20.001823,47335,7500.641870,595.828265,4755.725927,2149.087677,30438.112581,7119.582940,19780.332201,0.000015,1892.355281,466.049670,1181.319618,244.985992,400.171514,26.149556,248.238252,125.783706,19460.555741,5174.416696,12865.429555,...,0.511584,0.393858,0.517814,0.598009,0.002586,0.005788,0.001722,0.002694,0.911412,0.169381,0.651545,0.179751,0.098254,False,False,True,False,False,False,True,False,1.0,Alhambra,61897.39551,False
2,2,3,Arcadia,126258.98,12.228262,37169,13381.530815,2115.148625,6685.762725,4580.619465,8147.091902,2441.806335,4856.815380,0.000006,990.881657,334.528273,525.148452,131.204932,70.730779,0.017364,24.079301,46.634115,5085.192956,1626.462948,3048.032363,...,0.605520,0.596204,0.642072,0.495743,0.001375,0.000157,0.001546,0.002277,0.783141,0.222551,0.592985,0.185774,0.103864,False,False,True,False,False,False,True,False,2.0,Arcadia,92543.61666,True
3,3,4,Artesia,55463.91,3.000606,18036,2543.198368,249.844904,1424.499924,868.853540,6684.617774,1596.709299,4444.111268,0.000097,852.042661,301.822462,497.791440,52.428760,59.913886,0.003356,0.049661,59.860869,4909.630360,1361.455844,3131.351644,...,0.367121,0.321175,0.387681,0.341434,0.002986,0.000024,0.001575,0.012780,0.848194,0.201502,0.644574,0.153806,0.181542,False,False,False,False,False,False,True,False,3.0,Artesia,67486.31903,False
4,4,5,Avalon,11981.00,0.830664,3881,1298.646327,95.240319,830.798525,372.607483,1704.840285,714.150321,926.143353,0.000333,3.354092,0.000000,3.354092,0.000000,0.000000,0.000000,0.000000,0.000000,583.530943,294.597351,263.280451,...,0.000558,0.000000,0.000027,0.003757,0.000324,0.000000,0.000558,0.000000,0.574229,0.265365,0.580198,0.144410,0.067722,False,False,False,False,False,True,True,False,4.0,Avalon,73854.46872,True


In [32]:
disparity[disparity["name"] == "Culver City"]

,Unnamed: 0,id,name,tract,tract_percent,Unnamed: 0_x,total_white,white_17andunder,white_18to64,white_65andup,total_hispanic,hispanic_17andunder,hispanic_18to64,hispanic_65andup,total_black,black_17andunder,black_18to64,black_65andup,total_native_american,native_american_17andunder,native_american_18to64,native_american_65andup,total_other,other_17andunder,other_18to64,...,total_asian_percent,asian_17andunder_percent,asian_18to64_percent,asian_65andup_percent,total_pacificislander_percent,pacificislander_17andunder_percent,pacificislander_18to64_percent,pacificislander_65andup_percent,nonwhite_percentage,totalpop_17andunder_percent,totalpop_18to64_percent,totalpop_65andup_percent,healthcare_workers_percent,majority_nonwhite,majority_black,majority_asian,majority_white,majority_american_indian_and_alaska_native,majority_hispanic,majority_18to64,majority_65andup,Unnamed: 0_y,neighborhood,median_householdincome,majorityhigherthanmedianincomeLA
20,20,21,Culver City,137173.51,8.458713,43497,16715.265918,2562.476156,10728.876218,3423.913545,8845.68083,2074.155779,5565.435128,0.000033,2632.700334,482.814709,1660.39748,489.488145,122.931503,21.521738,81.624835,19.784931,4888.594953,1490.13727,2760.762196,...,0.157578,0.120852,0.173611,0.13863,0.0012,0.003086,0.000961,0.0,0.53318,0.189066,0.641709,0.168225,0.072984,False,False,False,False,False,False,True,False,21.0,Culver City,93492.86676,True


In [33]:
df_pivot[df_pivot["name"].str.contains("Culver City")]

,name,pct_week_9,pct_week_10,pct_week_11,pct_week_12,pct_week_13,pct_week_14,pct_week_15
21,Culver City,22.67,22.67,30.36,30.36,42.08,42.08,52.67


In [34]:
df_pivot["name"] = df_pivot["name"].str.strip(" ")

In [35]:
disparity_clean = disparity[
    [
        "name",
        "total_pop",
        #         "totalpop_17andunder_percent",
        #         "totalpop_18to64",
        #         "totalpop_65andup",
        "totalpop_17andunder_percent",
        "totalpop_65andup_percent",
        "nonwhite_percentage",
        "majority_white",
        "majority_hispanic",
        "median_householdincome",
    ]
]

In [36]:
df_merge = pd.merge(df_pivot, disparity_clean, on="name", indicator=True, how="left")

In [37]:
df_merge._merge.value_counts()

both          338
right_only      0
left_only       0
Name: _merge, dtype: int64

In [38]:
df_merge["change_week9_to_15"] = (
    ((df_merge["pct_week_15"] - df_merge["pct_week_9"]) / df_merge["pct_week_9"]) * 100
).round()

In [39]:
df_merge_slim = df_merge[
    [
        "name",
        "pct_week_9",
        "pct_week_15",
        "total_pop",
        "totalpop_17andunder_percent",
        "totalpop_65andup_percent",
        "nonwhite_percentage",
        "majority_white",
        "majority_hispanic",
        "median_householdincome",
        "change_week9_to_15",
    ]
].copy()

In [40]:
df_merge_slim[
    (df_merge_slim["pct_week_9"].notnull()) & (df_merge_slim["pct_week_15"].notnull())
].sort_values("change_week9_to_15", ascending=False).tail(10)

,name,pct_week_9,pct_week_15,total_pop,totalpop_17andunder_percent,totalpop_65andup_percent,nonwhite_percentage,majority_white,majority_hispanic,median_householdincome,change_week9_to_15
94,Los Angeles - Beverly Crest,32.56,54.03,10869.083843,0.196094,0.237505,0.134145,True,False,181638.58650,66.0
10,Beverly Hills,32.02,51.69,34265.801570,0.202520,0.213604,0.222052,True,False,109123.42940,61.0
327,Unincorporated - West LA,10.14,16.15,1154.826149,0.059281,0.181012,0.604410,False,False,56277.37386,59.0
64,Rolling Hills,29.97,46.77,1519.141044,0.188455,0.328955,0.287461,True,False,206440.31100,56.0
274,Unincorporated - Lakewood,18.27,28.44,448.309524,0.217250,0.132287,0.801341,False,False,88042.64006,56.0
93,Los Angeles - Bel Air,34.30,52.68,7089.875517,0.209939,0.303493,0.231194,True,False,200555.13980,54.0
306,Unincorporated - Santa Catalina Island,8.91,12.85,75.170250,0.085670,0.146566,0.328672,True,False,44555.34990,44.0
309,Unincorporated - Saugus/Canyon Country,39.16,50.60,387.871012,0.068194,0.201725,0.394058,True,False,86114.50759,29.0
288,Unincorporated - Padua Hills,32.03,38.12,23.307818,0.229093,0.231904,0.510892,False,False,200691.50320,19.0
4,Avalon,10.00,10.43,3050.104399,0.265365,0.144410,0.574229,False,True,73854.46872,4.0


In [41]:
df_merge_slim[df_merge_slim["name"].str.contains("Culver City")]

,name,pct_week_9,pct_week_15,total_pop,totalpop_17andunder_percent,totalpop_65andup_percent,nonwhite_percentage,majority_white,majority_hispanic,median_householdincome,change_week9_to_15
21,Culver City,22.67,52.67,35806.625282,0.189066,0.168225,0.53318,False,False,93492.86676,132.0


### Filter the dataframe to larger places

In [42]:
df_merge_slim["total_pop"] = df_merge_slim["total_pop"].fillna("").astype(int)

In [43]:
larger_places = df_merge_slim[df_merge_slim["total_pop"] > 5000].copy()

In [44]:
larger_places[larger_places["name"].str.contains("Culver")]

,name,pct_week_9,pct_week_15,total_pop,totalpop_17andunder_percent,totalpop_65andup_percent,nonwhite_percentage,majority_white,majority_hispanic,median_householdincome,change_week9_to_15
21,Culver City,22.67,52.67,35806,0.189066,0.168225,0.53318,False,False,93492.86676,132.0


### Which places with populations of more than 5,000 saw the largest pct increase from week 8 to week 15?

In [45]:
largest = (
    larger_places[
        (larger_places["pct_week_9"].notnull())
        & (larger_places["pct_week_15"].notnull())
    ]
    .sort_values("change_week9_to_15", ascending=False)
    .head(10)
)

In [46]:
largest

,name,pct_week_9,pct_week_15,total_pop,totalpop_17andunder_percent,totalpop_65andup_percent,nonwhite_percentage,majority_white,majority_hispanic,median_householdincome,change_week9_to_15
279,Unincorporated - Littlerock/Pearblossom,5.59,21.74,6786,0.284108,0.139551,0.700575,False,True,48752.94395,289.0
200,Los Angeles - University Park,6.46,25.15,25962,0.119700,0.061099,0.785405,False,False,24656.25697,289.0
194,Los Angeles - Thai Town,7.21,27.97,9314,0.107368,0.130282,0.527663,False,False,42919.83666,288.0
260,Unincorporated - Florence-Firestone,6.76,25.36,63373,0.317636,0.068662,0.995205,False,True,44489.45377,275.0
275,Unincorporated - Lennox,7.29,27.16,19025,0.285876,0.064121,0.983711,False,True,51161.26784,273.0
20,Cudahy,7.00,25.56,24767,0.305616,0.069818,0.969557,False,True,46504.81404,265.0
88,Los Angeles - Alsace,8.49,29.92,10053,0.209612,0.089146,0.939044,False,True,43093.10702,252.0
125,Los Angeles - Florence-Firestone,6.59,22.81,48257,0.319442,0.055256,0.994513,False,True,40483.50613,246.0
129,Los Angeles - Green Meadows,6.82,23.61,21718,0.308907,0.074795,0.994338,False,True,41366.22880,246.0
208,Los Angeles - Vernon Central,6.52,22.51,50398,0.323692,0.062574,0.996621,False,True,40567.66783,245.0


### Mean income and older population share in places with largest pct increases

In [47]:
round(
    larger_places[
        (larger_places["pct_week_9"].notnull())
        & (larger_places["pct_week_15"].notnull())
    ]
    .sort_values("change_week9_to_15", ascending=False)
    .head(10)["median_householdincome"]
    .mean()
)

42400

In [48]:
round(
    larger_places[
        (larger_places["pct_week_9"].notnull())
        & (larger_places["pct_week_15"].notnull())
    ]
    .sort_values("change_week9_to_15", ascending=False)
    .head(10)["totalpop_65andup_percent"]
    .mean()
    * 100,
    2,
)

8.15

### Mean income and older population share in places with smalles pct increases

In [49]:
round(
    larger_places[
        (larger_places["pct_week_9"].notnull())
        & (larger_places["pct_week_15"].notnull())
    ]
    .sort_values("change_week9_to_15", ascending=False)
    .tail(10)["median_householdincome"]
    .mean()
)

141297

In [50]:
round(
    larger_places[
        (larger_places["pct_week_9"].notnull())
        & (larger_places["pct_week_15"].notnull())
    ]
    .sort_values("change_week9_to_15", ascending=False)
    .tail(10)["totalpop_65andup_percent"]
    .mean()
    * 100,
    2,
)

22.77

### Which places with populations of more than 5,000 saw the smallest pct increase from week 8 to week 14?

In [51]:
smallest = (
    larger_places[
        (larger_places["pct_week_9"].notnull())
        & (larger_places["pct_week_15"].notnull())
    ]
    .sort_values("change_week9_to_15", ascending=False)
    .tail(10)
)

In [52]:
smallest

,name,pct_week_9,pct_week_15,total_pop,totalpop_17andunder_percent,totalpop_65andup_percent,nonwhite_percentage,majority_white,majority_hispanic,median_householdincome,change_week9_to_15
219,Los Angeles - Westwood,19.74,35.76,55923,0.088921,0.119817,0.488900,True,False,73222.64454,81.0
62,Rancho Palos Verdes,27.82,50.23,38343,0.213775,0.248590,0.501286,False,False,134930.47240,81.0
65,Rolling Hills Estates,31.19,55.39,9447,0.222468,0.247619,0.434340,True,False,165694.10080,78.0
57,Palos Verdes Estates,29.18,51.26,11153,0.213455,0.270924,0.342754,True,False,192218.47080,76.0
120,Los Angeles - Encino,30.77,54.11,48064,0.207176,0.199472,0.242364,True,False,96824.75157,76.0
103,Los Angeles - Century City,34.26,59.68,12434,0.177379,0.234012,0.316788,True,False,119661.69920,74.0
106,Los Angeles - Cheviot Hills,37.08,63.66,8640,0.211283,0.202283,0.269950,True,False,139097.53580,72.0
94,Los Angeles - Beverly Crest,32.56,54.03,10869,0.196094,0.237505,0.134145,True,False,181638.58650,66.0
10,Beverly Hills,32.02,51.69,34265,0.202520,0.213604,0.222052,True,False,109123.42940,61.0
93,Los Angeles - Bel Air,34.30,52.68,7089,0.209939,0.303493,0.231194,True,False,200555.13980,54.0


### Which places have the highest vaccination coverage right now?

In [53]:
larger_places[
    (larger_places["pct_week_9"].notnull()) & (larger_places["pct_week_15"].notnull())
][
    [
        "name",
        "pct_week_15",
        "majority_hispanic",
        "totalpop_17andunder_percent",
        "totalpop_65andup_percent",
    ]
].sort_values(
    "pct_week_15", ascending=False
).head(
    10
)

,name,pct_week_15,majority_hispanic,totalpop_17andunder_percent,totalpop_65andup_percent
106,Los Angeles - Cheviot Hills,63.66,False,0.211283,0.202283
175,Los Angeles - Rancho Park,62.71,False,0.207814,0.163806
103,Los Angeles - Century City,59.68,False,0.177379,0.234012
165,Los Angeles - Pacific Palisades,56.17,False,0.248622,0.236204
173,Los Angeles - Playa Vista,55.74,False,0.186668,0.104989
65,Rolling Hills Estates,55.39,False,0.222468,0.247619
120,Los Angeles - Encino,54.11,False,0.207176,0.199472
94,Los Angeles - Beverly Crest,54.03,False,0.196094,0.237505
74,Sierra Madre,53.69,False,0.182263,0.228925
70,San Marino,53.29,False,0.227649,0.206610


### Which places have the lowest vaccination coverage right now? 

In [54]:
larger_places[
    (larger_places["pct_week_9"].notnull()) & (larger_places["pct_week_15"].notnull())
][
    [
        "name",
        "pct_week_15",
        "majority_hispanic",
        "totalpop_17andunder_percent",
        "totalpop_65andup_percent",
    ]
].sort_values(
    "pct_week_15", ascending=False
).tail(
    10
)

,name,pct_week_15,majority_hispanic,totalpop_17andunder_percent,totalpop_65andup_percent
218,Los Angeles - Westlake,22.88,True,0.230150,0.093303
125,Los Angeles - Florence-Firestone,22.81,True,0.319442,0.055256
208,Los Angeles - Vernon Central,22.51,True,0.323692,0.062574
185,Los Angeles - South Park,22.28,True,0.322496,0.056965
102,Los Angeles - Central,22.14,True,0.306173,0.065168
104,Los Angeles - Century Palms/Cove,21.88,True,0.311102,0.073659
279,Unincorporated - Littlerock/Pearblossom,21.74,True,0.284108,0.139551
211,Los Angeles - Watts,21.42,True,0.358358,0.055530
207,Los Angeles - Vermont Vista,20.70,True,0.319345,0.067000
272,Unincorporated - Lake Los Angeles,19.11,True,0.302845,0.114554


### Merge the places with the largest and smallest increases in vax coverage for graphics

In [55]:
largest["category"] = "Largest increase"
smallest["category"] = "Smallest increase"

In [56]:
large_small = pd.concat([largest, smallest]).reset_index()

In [57]:
large_small.head()

,index,name,pct_week_9,pct_week_15,total_pop,totalpop_17andunder_percent,totalpop_65andup_percent,nonwhite_percentage,majority_white,majority_hispanic,median_householdincome,change_week9_to_15,category
0,279,Unincorporated - Littlerock/Pearblossom,5.59,21.74,6786,0.284108,0.139551,0.700575,False,True,48752.94395,289.0,Largest increase
1,200,Los Angeles - University Park,6.46,25.15,25962,0.119700,0.061099,0.785405,False,False,24656.25697,289.0,Largest increase
2,194,Los Angeles - Thai Town,7.21,27.97,9314,0.107368,0.130282,0.527663,False,False,42919.83666,288.0,Largest increase
3,260,Unincorporated - Florence-Firestone,6.76,25.36,63373,0.317636,0.068662,0.995205,False,True,44489.45377,275.0,Largest increase
4,275,Unincorporated - Lennox,7.29,27.16,19025,0.285876,0.064121,0.983711,False,True,51161.26784,273.0,Largest increase


In [58]:
large_small["name_clean"] = (
    large_small["name"]
    .str.replace("City of", "", regex=False)
    .str.replace("Los Angeles - ", "", regex=False)
    .str.replace("Unincorporated - ", "", regex=False)
)

In [59]:
large_small_slim = large_small[
    [
        "category",
        "name_clean",
        "pct_week_9",
        "pct_week_15",
        "change_week9_to_15",
    ]
].copy()

In [60]:
large_small_slim.rename(
    columns={
        "change_week9_to_15": "% change",
        "pct_week_15": "Week 15",
        "pct_week_9": "Week 9",
        "name_clean": "Place",
        "category": "Category",
    },
    inplace=True,
)

In [61]:
large_small_slim.to_csv("output/vaccine-community/large_small_slim.csv", index=False)

---

### Merge with geography

In [62]:
places_geo = gpd.read_file(
    "input/cities-neighborhoods-unincorporated-la-county-no-islands.geojson"
)

In [63]:
places_geo.head()

,NAME,geometry
0,Agoura Hills,"MULTIPOLYGON (((117201.354 -426752.672, 116472..."
1,Bellflower,"MULTIPOLYGON (((174818.350 -454427.122, 174810..."
2,Beverly Hills,"MULTIPOLYGON (((148374.547 -432436.704, 148309..."
3,Burbank,"MULTIPOLYGON (((156006.516 -426897.025, 156303..."
4,Calabasas,"MULTIPOLYGON (((120408.368 -426638.916, 120344..."


### Clean up places names

In [64]:
larger_places["name_clean"] = (
    larger_places["name"]
    .str.strip(" ")
    .str.replace("City of", "", regex=False)
    .str.replace("Los Angeles - ", "", regex=False)
    .str.replace("Unincorporated - ", "", regex=False)
)

In [65]:
larger_places.head()

,name,pct_week_9,pct_week_15,total_pop,totalpop_17andunder_percent,totalpop_65andup_percent,nonwhite_percentage,majority_white,majority_hispanic,median_householdincome,change_week9_to_15,name_clean
0,Agoura Hills,21.82,47.18,19234,0.215044,0.159371,0.256824,True,False,121292.28270,116.0,Agoura Hills
1,Alhambra,18.68,45.10,84668,0.169381,0.179751,0.911412,False,False,61897.39551,141.0,Alhambra
2,Arcadia,20.90,46.08,61706,0.222551,0.185774,0.783141,False,False,92543.61666,120.0,Arcadia
3,Artesia,20.83,42.27,16752,0.201502,0.153806,0.848194,False,False,67486.31903,103.0,Artesia
5,Azusa,12.60,31.45,45944,0.227478,0.103444,0.834163,False,True,70019.22957,150.0,Azusa


In [66]:
len(larger_places)

240

In [67]:
larger_places[larger_places["name_clean"].str.contains("Beverly")]

,name,pct_week_9,pct_week_15,total_pop,totalpop_17andunder_percent,totalpop_65andup_percent,nonwhite_percentage,majority_white,majority_hispanic,median_householdincome,change_week9_to_15,name_clean
10,Beverly Hills,32.02,51.69,34265,0.202520,0.213604,0.222052,True,False,109123.4294,61.0,Beverly Hills
94,Los Angeles - Beverly Crest,32.56,54.03,10869,0.196094,0.237505,0.134145,True,False,181638.5865,66.0,Beverly Crest
95,Los Angeles - Beverlywood,26.20,52.00,11511,0.251519,0.136678,0.247145,True,False,105509.1624,98.0,Beverlywood


In [68]:
df_geo = places_geo.merge(larger_places, left_on="NAME", right_on="name_clean")

In [69]:
df_geo.head()

,NAME,geometry,name,pct_week_9,pct_week_15,total_pop,totalpop_17andunder_percent,totalpop_65andup_percent,nonwhite_percentage,majority_white,majority_hispanic,median_householdincome,change_week9_to_15,name_clean
0,Agoura Hills,"MULTIPOLYGON (((117201.354 -426752.672, 116472...",Agoura Hills,21.82,47.18,19234,0.215044,0.159371,0.256824,True,False,121292.28270,116.0,Agoura Hills
1,Bellflower,"MULTIPOLYGON (((174818.350 -454427.122, 174810...",Bellflower,13.14,32.12,77307,0.254948,0.110609,0.841269,False,True,60014.00406,144.0,Bellflower
2,Beverly Hills,"MULTIPOLYGON (((148374.547 -432436.704, 148309...",Beverly Hills,32.02,51.69,34265,0.202520,0.213604,0.222052,True,False,109123.42940,61.0,Beverly Hills
3,Burbank,"MULTIPOLYGON (((156006.516 -426897.025, 156303...",Burbank,15.93,40.17,103694,0.181871,0.154310,0.430812,True,False,75896.99690,152.0,Burbank
4,Calabasas,"MULTIPOLYGON (((120408.368 -426638.916, 120344...",Calabasas,26.78,50.52,14552,0.233885,0.207053,0.219005,True,False,140091.43750,89.0,Calabasas


In [70]:
len(df_geo)

269

In [71]:
df_geo_slim = df_geo[
    ["name_clean", "pct_week_9", "pct_week_15", "change_week9_to_15", "geometry"]
]

In [72]:
df_geo_slim.to_file("output/vaccine-community/df_geo.geojson", driver="GeoJSON")

---

### Export to csv

In [73]:
import datetime as dt

today = dt.datetime.today().strftime("%m-%d-%Y")

In [74]:
concatenated_df.to_csv(
    "output/vaccine-community/lac_vax_by_place_raw_table_" + today + ".csv",
    index=False,
)

In [75]:
df_pivot.to_csv(
    "output/vaccine-community/lac_vax_by_place_weeks_" + today + ".csv",
    index=False,
)

In [76]:
larger_places[
    (larger_places["pct_week_9"].notnull()) & (larger_places["pct_week_15"].notnull())
].sort_values("change_week9_to_15", ascending=False).head(10).to_csv(
    "output/vaccine-community/largest_vax_coverage_change_" + today + ".csv",
    index=False,
)

In [77]:
larger_places[
    (larger_places["pct_week_9"].notnull()) & (larger_places["pct_week_15"].notnull())
].sort_values("change_week9_to_15", ascending=False).tail(10).to_csv(
    "output/vaccine-community/smallest_vax_coverage_change_" + today + ".csv",
    index=False,
)

In [78]:
larger_places[
    (larger_places["pct_week_9"].notnull()) & (larger_places["pct_week_15"].notnull())
][["name", "pct_week_15", "total_pop"]].sort_values(
    "pct_week_15", ascending=False
).tail(
    10
).to_csv(
    "output/vaccine-community/lowest_coverage_" + today + ".csv",
    index=False,
)

In [79]:
larger_places[
    (larger_places["pct_week_9"].notnull()) & (larger_places["pct_week_15"].notnull())
][["name", "pct_week_15", "total_pop"]].sort_values(
    "pct_week_15", ascending=False
).head(
    10
).to_csv(
    "output/vaccine-community/highest_coverage_" + today + ".csv",
    index=False,
)